In [0]:
dbutils.widgets.dropdown("env_stage", "dev", ["dev", "prod"], "Pipeline stage")

dbutils.widgets.dropdown("exclude_pms", "False", ["True", "False"], "Exclude PMS")

dbutils.widgets.dropdown("target_type", "REVENUE", ["REVENUE", "ROOMS"], "Target Type")

dbutils.widgets.text("selected_hotels", "", "Hotels")

dbutils.widgets.dropdown("cache_models", "False", ["True", "False"], "Run Cache Models")

In [0]:
import requests

def str_to_bool(value):
  FALSE_VALUES = ['false', 'no', '0']
  TRUE_VALUES = ['true', 'yes', '1']
  lvalue = str(value).lower()
  if lvalue in (FALSE_VALUES): return False
  if lvalue in (TRUE_VALUES):  return True
  raise Exception("String value should be one of {}, but got '{}'.".format(FALSE_VALUES + TRUE_VALUES, value))

def extract_selected_hotels(value):
    if value == "":
        return []
    elif "," in value:
        hotels = value.split(",")
        return hotels

    return [value]

In [0]:

selected_hotels_str = getArgument("selected_hotels")
selected_hotels = extract_selected_hotels(selected_hotels_str)
env = getArgument("env_stage")

if not len(selected_hotels)==0:
    databricks_instance = "adb-4286524806154993.13.azuredatabricks.net"
    scope = f"phg-data-{env}-dbsp-dbxbackend"
    restapi = f"phg-data-{env}-scrt-api"

    ACCESS_TOKEN  = dbutils.secrets.get(scope = scope, key =restapi)
    JOB_ID = "880810573214147"

    api_url = f"https://{databricks_instance}/api/2.1/jobs/run-now"

    headers = {
        "Authorization": f"Bearer {ACCESS_TOKEN}",
        "Content-Type": "application/json"
    }

    job_payload = {
        "job_id": JOB_ID,
        "notebook_params": {
            "selected_hotels": selected_hotels_str,
        }
    }

    response = requests.post(api_url, headers=headers, json=job_payload)
    print(response)

    # Check if the request was successful
    if response.status_code == 200:
        response_json = response.json()
        print("Job triggered successfully!")
        print(f"Run ID: {response_json['run_id']}")
    else:
        response_json = response.json()
        print("Failed to trigger job.")
        print(f"Error message: {response_json['error_message']}")

else:
    raise ValueError("No selected hotels were provided. Please run the model cache script manually")